In [1]:
import pandas as pd
import numpy as np

sdoh = pd.read_csv("final_df.csv")
df = pd.read_csv('AD_entities.csv')

In [2]:
sdohlist = sdoh['text'].tolist()
len(sdohlist)

5290

In [3]:
df['ENTITIES'] = df['ENTITIES'].apply(lambda x: [item.strip().strip("'") for item in x[1:-1].split(',')])

In [4]:
len(df['ENTITIES'].values[0])

528

In [5]:
# for each row in the dataframe, check if the entity is in the sdoh list
# if not, remove it from the list
for i in range(len(df)):
    df['ENTITIES'].values[i] = [item for item in df['ENTITIES'].values[i] if item in sdohlist]

len(df['ENTITIES'].values[0])

57

In [6]:
# remove columns 'DOB', 'ADMITTIME', 'CATEGORY', 'TEXT', 'TEXT_CLEAN'
df = df.drop(['DOB', 'ADMITTIME', 'CATEGORY', 'TEXT', 'TEXT_CLEAN'], axis=1)
df

,SUBJECT_ID,GENDER,INSURANCE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_CODE,DRUG,AGE,ENTITIES
0,157,['M'],['Medicare'],['CATHOLIC'],['SINGLE'],['WHITE'],"['43889', '78079', 'V5843', '25000', '7310', '...","['SW', 'Magnesium Sulfate', 'Sodium Chloride 0...",80.0,"[medical history, tablets, peg tube placement,..."
1,187,['F'],['Medicare'],['PROTESTANT QUAKER'],['DIVORCED'],['WHITE'],"['0383', '9992', '41071', '496', '2762', '4518...",NaN,66.0,"[primary care physician, blood cultures, drink..."
2,253,['F'],['Medicare'],['CATHOLIC'],['WIDOWED'],['WHITE'],"['4260', '486', '4240', '2449', '3310', '29410...","['Ipratropium Bromide MDI', 'Albuterol', 'Iso-...",84.0,"[medical history, discharge, family history, h..."
3,275,['M'],['Medicare'],['NOT SPECIFIED'],['MARRIED'],['WHITE'],"['0389', '51881', '78552', '5849', '4538', '28...","['Cosyntropin', 'Magnesium Sulfate', 'Phytonad...",82.0,"[hospital sdh, surgical changes, medical histo..."
4,345,['F'],['Medicare'],['CATHOLIC'],['MARRIED'],['HISPANIC OR LATINO'],"['78039', '42823', '34830', '2762', '2903', '4...","['Metoprolol Tartrate', 'Furosemide', 'LeVETir...",89+,"[atrial fibrillation, medical history, family ..."
...,...,...,...,...,...,...,...,...,...,...
681,99417,['M'],['Medicare'],['NOT SPECIFIED'],['MARRIED'],['WHITE'],"['0389', '78552', '5570', '51881', '2762', '56...","['Sodium Chloride 0.9% Flush', '5% Dextrose',...",87,"[medical history, family history, surgical eva..."
682,99439,['M'],['Medicare'],['PROTESTANT QUAKER'],['MARRIED'],['BLACK/AFRICAN AMERICAN'],"['34830', '42842', '5849', '4168', '43883', '4...","['Potassium Chloride', '0.9% Sodium Chloride',...",84,"[eye discharge medications, hospice, exuberant..."
683,99454,['F'],['Medicare'],['PROTESTANT QUAKER'],['MARRIED'],['WHITE'],"['41519', '51881', '45342', '2930', '3310', '2...","['Omeprazole', 'Namenda', 'Gabapentin', 'Levot...",79,"[warfarin 1 mg, department radiology, treat, o..."
684,99864,['M'],['Medicare'],['JEWISH'],['SINGLE'],['WHITE'],"['5780', '43820', '78791', '7873', 'V441', '79...","['1/2 NS', 'Insulin', 'Lisinopril', '0.9% Sodi...",83,"[primary care physician, calcified granuloma r..."


In [7]:
import ast

# Step 1: Extract value from [' '] for specific columns
def extract_value_from_list(value):
    try:
        # Safely evaluate the string to a Python list and extract the first element
        return ast.literal_eval(value)[0]
    except:
        # If evaluation fails, return the original value
        return value

# Columns to clean
columns_to_clean = ['GENDER', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY']
for col in columns_to_clean:
    df[col] = df[col].apply(extract_value_from_list)

# Step 2: Convert 'ICD9_CODE' and 'DRUG' from string to list
def convert_to_list(value):
    try:
        # Safely evaluate the string to a Python list
        return ast.literal_eval(value)
    except:
        # If evaluation fails, wrap the value in a list
        return [value]

for col in ['ICD9_CODE', 'DRUG']:
    df[col] = df[col].apply(convert_to_list)

# Step 3: Convert AGE to integer, skip '89+' and other non-numeric values
def convert_age(value):
    try:
        if isinstance(value, str) and '+' in value:
            return value
        return int(float(value))  # Convert to integer
    except:
        return None  # Return None if conversion fails

df['AGE'] = df['AGE'].apply(convert_age)

# Display the modified DataFrame
df


,SUBJECT_ID,GENDER,INSURANCE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_CODE,DRUG,AGE,ENTITIES
0,157,M,Medicare,CATHOLIC,SINGLE,WHITE,"[43889, 78079, V5843, 25000, 7310, 2768, 7872,...","[SW, Magnesium Sulfate, Sodium Chloride 0.9% ...",80,"[medical history, tablets, peg tube placement,..."
1,187,F,Medicare,PROTESTANT QUAKER,DIVORCED,WHITE,"[0383, 9992, 41071, 496, 2762, 45182, 2768, 33...",[nan],66,"[primary care physician, blood cultures, drink..."
2,253,F,Medicare,CATHOLIC,WIDOWED,WHITE,"[4260, 486, 4240, 2449, 3310, 29410, 4168, 272...","[Ipratropium Bromide MDI, Albuterol, Iso-Osmot...",84,"[medical history, discharge, family history, h..."
3,275,M,Medicare,NOT SPECIFIED,MARRIED,WHITE,"[0389, 51881, 78552, 5849, 4538, 2869, 85221, ...","[Cosyntropin, Magnesium Sulfate, Phytonadione,...",82,"[hospital sdh, surgical changes, medical histo..."
4,345,F,Medicare,CATHOLIC,MARRIED,HISPANIC OR LATINO,"[78039, 42823, 34830, 2762, 2903, 4280, 42731,...","[Metoprolol Tartrate, Furosemide, LeVETiraceta...",89+,"[atrial fibrillation, medical history, family ..."
...,...,...,...,...,...,...,...,...,...,...
681,99417,M,Medicare,NOT SPECIFIED,MARRIED,WHITE,"[0389, 78552, 5570, 51881, 2762, 5609, 99592, ...","[Sodium Chloride 0.9% Flush, 5% Dextrose, Sol...",87,"[medical history, family history, surgical eva..."
682,99439,M,Medicare,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,"[34830, 42842, 5849, 4168, 43883, 496, 32723, ...","[Potassium Chloride, 0.9% Sodium Chloride, Iso...",84,"[eye discharge medications, hospice, exuberant..."
683,99454,F,Medicare,PROTESTANT QUAKER,MARRIED,WHITE,"[41519, 51881, 45342, 2930, 3310, 29410, 2449,...","[Omeprazole, Namenda, Gabapentin, Levothyroxin...",79,"[warfarin 1 mg, department radiology, treat, o..."
684,99864,M,Medicare,JEWISH,SINGLE,WHITE,"[5780, 43820, 78791, 7873, V441, 7921, 9916, 2...","[1/2 NS, Insulin, Lisinopril, 0.9% Sodium Chlo...",83,"[primary care physician, calcified granuloma r..."


In [9]:
# Get the unique values of specified columns, skipping missing ones
columns_to_check = ['GENDER', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'AGE']

# Create a dictionary to store the unique values for each column (if it exists)
unique_values = {
    col: df[col].unique().tolist() for col in columns_to_check if col in df.columns
}

# Display the unique values
unique_values


{'GENDER': ['M', 'F'],
 'INSURANCE': ['Medicare', 'Private', 'Medicaid'],
 'RELIGION': ['CATHOLIC',
  'PROTESTANT QUAKER',
  'NOT SPECIFIED',
  'OTHER',
  'JEWISH',
  'UNOBTAINABLE',
  'ROMANIAN EAST. ORTH',
  "JEHOVAH'S WITNESS",
  'EPISCOPALIAN',
  'GREEK ORTHODOX',
  '[nan]',
  'MUSLIM',
  'UNITARIAN-UNIVERSALIST',
  'CHRISTIAN SCIENTIST',
  'BUDDHIST',
  '7TH DAY ADVENTIST'],
 'MARITAL_STATUS': ['SINGLE',
  'DIVORCED',
  'WIDOWED',
  'MARRIED',
  'SEPARATED',
  '[nan]',
  'UNKNOWN (DEFAULT)'],
 'ETHNICITY': ['WHITE',
  'HISPANIC OR LATINO',
  'UNKNOWN/NOT SPECIFIED',
  'ASIAN',
  'BLACK/AFRICAN AMERICAN',
  'HISPANIC/LATINO - PUERTO RICAN',
  'WHITE - RUSSIAN',
  'OTHER',
  'BLACK/CAPE VERDEAN',
  'UNABLE TO OBTAIN',
  'ASIAN - FILIPINO',
  'AMERICAN INDIAN/ALASKA NATIVE',
  'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
  'ASIAN - CHINESE',
  'MIDDLE EASTERN',
  'BLACK/HAITIAN',
  'MULTI RACE ETHNICITY',
  'PORTUGUESE'],
 'AGE': [80,
  66,
  84,
  82,
  '89+',
  71,
  83,
  87,
  78

In [10]:
# if religion is 'NOT SPECIFIED', 'OTHER', 'UNOBTAINABLE', '[nan]', replace with 'UNKNOWN'
df['RELIGION'] = df['RELIGION'].replace(['NOT SPECIFIED', 'OTHER', 'UNOBTAINABLE', '[nan]'], 'UNKNOWN')
df['MARITAL_STATUS'] = df['MARITAL_STATUS'].replace(['UNKNOWN (DEFAULT)', '[nan]'], 'UNKNOWN')
df['ETHNICITY'] = df['ETHNICITY'].replace(['UNKNOWN/NOT SPECIFIED', 'OTHER', 'UNABLE TO OBTAIN'], 'UNKNOWN')

In [11]:
df

,SUBJECT_ID,GENDER,INSURANCE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_CODE,DRUG,AGE,ENTITIES
0,157,M,Medicare,CATHOLIC,SINGLE,WHITE,"[43889, 78079, V5843, 25000, 7310, 2768, 7872,...","[SW, Magnesium Sulfate, Sodium Chloride 0.9% ...",80,"[medical history, tablets, peg tube placement,..."
1,187,F,Medicare,PROTESTANT QUAKER,DIVORCED,WHITE,"[0383, 9992, 41071, 496, 2762, 45182, 2768, 33...",[nan],66,"[primary care physician, blood cultures, drink..."
2,253,F,Medicare,CATHOLIC,WIDOWED,WHITE,"[4260, 486, 4240, 2449, 3310, 29410, 4168, 272...","[Ipratropium Bromide MDI, Albuterol, Iso-Osmot...",84,"[medical history, discharge, family history, h..."
3,275,M,Medicare,UNKNOWN,MARRIED,WHITE,"[0389, 51881, 78552, 5849, 4538, 2869, 85221, ...","[Cosyntropin, Magnesium Sulfate, Phytonadione,...",82,"[hospital sdh, surgical changes, medical histo..."
4,345,F,Medicare,CATHOLIC,MARRIED,HISPANIC OR LATINO,"[78039, 42823, 34830, 2762, 2903, 4280, 42731,...","[Metoprolol Tartrate, Furosemide, LeVETiraceta...",89+,"[atrial fibrillation, medical history, family ..."
...,...,...,...,...,...,...,...,...,...,...
681,99417,M,Medicare,UNKNOWN,MARRIED,WHITE,"[0389, 78552, 5570, 51881, 2762, 5609, 99592, ...","[Sodium Chloride 0.9% Flush, 5% Dextrose, Sol...",87,"[medical history, family history, surgical eva..."
682,99439,M,Medicare,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,"[34830, 42842, 5849, 4168, 43883, 496, 32723, ...","[Potassium Chloride, 0.9% Sodium Chloride, Iso...",84,"[eye discharge medications, hospice, exuberant..."
683,99454,F,Medicare,PROTESTANT QUAKER,MARRIED,WHITE,"[41519, 51881, 45342, 2930, 3310, 29410, 2449,...","[Omeprazole, Namenda, Gabapentin, Levothyroxin...",79,"[warfarin 1 mg, department radiology, treat, o..."
684,99864,M,Medicare,JEWISH,SINGLE,WHITE,"[5780, 43820, 78791, 7873, V441, 7921, 9916, 2...","[1/2 NS, Insulin, Lisinopril, 0.9% Sodium Chlo...",83,"[primary care physician, calcified granuloma r..."


In [12]:
# replace all the [nan] and [] with np.nan
df = df.replace(['[nan]', '[]'], np.nan)
df

,SUBJECT_ID,GENDER,INSURANCE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_CODE,DRUG,AGE,ENTITIES
0,157,M,Medicare,CATHOLIC,SINGLE,WHITE,"[43889, 78079, V5843, 25000, 7310, 2768, 7872,...","[SW, Magnesium Sulfate, Sodium Chloride 0.9% ...",80,"[medical history, tablets, peg tube placement,..."
1,187,F,Medicare,PROTESTANT QUAKER,DIVORCED,WHITE,"[0383, 9992, 41071, 496, 2762, 45182, 2768, 33...",[nan],66,"[primary care physician, blood cultures, drink..."
2,253,F,Medicare,CATHOLIC,WIDOWED,WHITE,"[4260, 486, 4240, 2449, 3310, 29410, 4168, 272...","[Ipratropium Bromide MDI, Albuterol, Iso-Osmot...",84,"[medical history, discharge, family history, h..."
3,275,M,Medicare,UNKNOWN,MARRIED,WHITE,"[0389, 51881, 78552, 5849, 4538, 2869, 85221, ...","[Cosyntropin, Magnesium Sulfate, Phytonadione,...",82,"[hospital sdh, surgical changes, medical histo..."
4,345,F,Medicare,CATHOLIC,MARRIED,HISPANIC OR LATINO,"[78039, 42823, 34830, 2762, 2903, 4280, 42731,...","[Metoprolol Tartrate, Furosemide, LeVETiraceta...",89+,"[atrial fibrillation, medical history, family ..."
...,...,...,...,...,...,...,...,...,...,...
681,99417,M,Medicare,UNKNOWN,MARRIED,WHITE,"[0389, 78552, 5570, 51881, 2762, 5609, 99592, ...","[Sodium Chloride 0.9% Flush, 5% Dextrose, Sol...",87,"[medical history, family history, surgical eva..."
682,99439,M,Medicare,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,"[34830, 42842, 5849, 4168, 43883, 496, 32723, ...","[Potassium Chloride, 0.9% Sodium Chloride, Iso...",84,"[eye discharge medications, hospice, exuberant..."
683,99454,F,Medicare,PROTESTANT QUAKER,MARRIED,WHITE,"[41519, 51881, 45342, 2930, 3310, 29410, 2449,...","[Omeprazole, Namenda, Gabapentin, Levothyroxin...",79,"[warfarin 1 mg, department radiology, treat, o..."
684,99864,M,Medicare,JEWISH,SINGLE,WHITE,"[5780, 43820, 78791, 7873, V441, 7921, 9916, 2...","[1/2 NS, Insulin, Lisinopril, 0.9% Sodium Chlo...",83,"[primary care physician, calcified granuloma r..."


In [19]:
# Function to replace [nan] and [] with np.nan
def replace_empty_or_nan_lists(value):
    if isinstance(value, list) and (len(value) == 0 or all(pd.isna(x) for x in value)):
        return np.nan
    return value

# Apply the function to relevant columns
columns_to_replace = ['ICD9_CODE', 'DRUG', 'ENTITIES']
for col in columns_to_replace:
    df[col] = df[col].apply(replace_empty_or_nan_lists)

# Display the DataFrame to verify the changes
df

,SUBJECT_ID,GENDER,INSURANCE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_CODE,DRUG,AGE,ENTITIES
0,157,M,Medicare,CATHOLIC,SINGLE,WHITE,"[43889, 78079, V5843, 25000, 7310, 2768, 7872,...","[SW, Magnesium Sulfate, Sodium Chloride 0.9% ...",80,"[medical history, tablets, peg tube placement,..."
1,187,F,Medicare,PROTESTANT QUAKER,DIVORCED,WHITE,"[0383, 9992, 41071, 496, 2762, 45182, 2768, 33...",NaN,66,"[primary care physician, blood cultures, drink..."
2,253,F,Medicare,CATHOLIC,WIDOWED,WHITE,"[4260, 486, 4240, 2449, 3310, 29410, 4168, 272...","[Ipratropium Bromide MDI, Albuterol, Iso-Osmot...",84,"[medical history, discharge, family history, h..."
3,275,M,Medicare,UNKNOWN,MARRIED,WHITE,"[0389, 51881, 78552, 5849, 4538, 2869, 85221, ...","[Cosyntropin, Magnesium Sulfate, Phytonadione,...",82,"[hospital sdh, surgical changes, medical histo..."
4,345,F,Medicare,CATHOLIC,MARRIED,HISPANIC OR LATINO,"[78039, 42823, 34830, 2762, 2903, 4280, 42731,...","[Metoprolol Tartrate, Furosemide, LeVETiraceta...",89+,"[atrial fibrillation, medical history, family ..."
...,...,...,...,...,...,...,...,...,...,...
681,99417,M,Medicare,UNKNOWN,MARRIED,WHITE,"[0389, 78552, 5570, 51881, 2762, 5609, 99592, ...","[Sodium Chloride 0.9% Flush, 5% Dextrose, Sol...",87,"[medical history, family history, surgical eva..."
682,99439,M,Medicare,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,"[34830, 42842, 5849, 4168, 43883, 496, 32723, ...","[Potassium Chloride, 0.9% Sodium Chloride, Iso...",84,"[eye discharge medications, hospice, exuberant..."
683,99454,F,Medicare,PROTESTANT QUAKER,MARRIED,WHITE,"[41519, 51881, 45342, 2930, 3310, 29410, 2449,...","[Omeprazole, Namenda, Gabapentin, Levothyroxin...",79,"[warfarin 1 mg, department radiology, treat, o..."
684,99864,M,Medicare,JEWISH,SINGLE,WHITE,"[5780, 43820, 78791, 7873, V441, 7921, 9916, 2...","[1/2 NS, Insulin, Lisinopril, 0.9% Sodium Chlo...",83,"[primary care physician, calcified granuloma r..."


In [22]:
# lowercase everything in the dataframe including column names
df = df.applymap(lambda s:s.lower() if type(s) == str else s)
df.columns = df.columns.str.lower()
df

,subject_id,gender,insurance,religion,marital_status,ethnicity,icd9_code,drug,age,entities
0,157,m,medicare,catholic,single,white,"[43889, 78079, V5843, 25000, 7310, 2768, 7872,...","[SW, Magnesium Sulfate, Sodium Chloride 0.9% ...",80,"[medical history, tablets, peg tube placement,..."
1,187,f,medicare,protestant quaker,divorced,white,"[0383, 9992, 41071, 496, 2762, 45182, 2768, 33...",NaN,66,"[primary care physician, blood cultures, drink..."
2,253,f,medicare,catholic,widowed,white,"[4260, 486, 4240, 2449, 3310, 29410, 4168, 272...","[Ipratropium Bromide MDI, Albuterol, Iso-Osmot...",84,"[medical history, discharge, family history, h..."
3,275,m,medicare,unknown,married,white,"[0389, 51881, 78552, 5849, 4538, 2869, 85221, ...","[Cosyntropin, Magnesium Sulfate, Phytonadione,...",82,"[hospital sdh, surgical changes, medical histo..."
4,345,f,medicare,catholic,married,hispanic or latino,"[78039, 42823, 34830, 2762, 2903, 4280, 42731,...","[Metoprolol Tartrate, Furosemide, LeVETiraceta...",89+,"[atrial fibrillation, medical history, family ..."
...,...,...,...,...,...,...,...,...,...,...
681,99417,m,medicare,unknown,married,white,"[0389, 78552, 5570, 51881, 2762, 5609, 99592, ...","[Sodium Chloride 0.9% Flush, 5% Dextrose, Sol...",87,"[medical history, family history, surgical eva..."
682,99439,m,medicare,protestant quaker,married,black/african american,"[34830, 42842, 5849, 4168, 43883, 496, 32723, ...","[Potassium Chloride, 0.9% Sodium Chloride, Iso...",84,"[eye discharge medications, hospice, exuberant..."
683,99454,f,medicare,protestant quaker,married,white,"[41519, 51881, 45342, 2930, 3310, 29410, 2449,...","[Omeprazole, Namenda, Gabapentin, Levothyroxin...",79,"[warfarin 1 mg, department radiology, treat, o..."
684,99864,m,medicare,jewish,single,white,"[5780, 43820, 78791, 7873, V441, 7921, 9916, 2...","[1/2 NS, Insulin, Lisinopril, 0.9% Sodium Chlo...",83,"[primary care physician, calcified granuloma r..."


In [30]:
# save the dataframe to a temp.csv
df.to_csv('temp.csv', index=False)

In [27]:
# # create new df to store KG
# mimicKG = pd.DataFrame(columns=['head', 'head_type', 'tail', 'tail_type', 'relation'])

# # head = gender, tail = each element in the icd9_code list
# rows = []
# for i in range(len(df)):
#     for item in df['icd9_code'].values[i]:
#         new_row = {
#             'head': df['gender'].values[i],
#             'head_type': 'gender',
#             'tail': item,
#             'tail_type': 'disease',
#             'relation': 'gender_disease'
#         }
#         rows.append(new_row)
# mimicKG = pd.concat([mimicKG, pd.DataFrame(rows)], ignore_index=True)

# rows = []
# for i in range(len(df)):
#     for item in df['icd9_code'].values[i]:
#         new_row = {
#             'head': df['insurance'].values[i],
#             'head_type': 'insurance',
#             'tail': item,
#             'tail_type': 'disease',
#             'relation': 'insurance_disease'
#         }
#         rows.append(new_row)
# mimicKG = pd.concat([mimicKG, pd.DataFrame(rows)], ignore_index=True)

# # create a new column 'weight', if head and tail are the same, add 1 to the weight, remove duplicates
# mimicKG['weight'] = 1
# mimicKG = mimicKG.groupby(['head', 'head_type', 'tail', 'tail_type', 'relation']).size().reset_index(name='weight')
# mimicKG

In [38]:
# Create new DataFrame to store KG
mimicKG = pd.DataFrame(columns=['head', 'head_type', 'tail', 'tail_type', 'relation'])

# List to store all rows before concatenating
rows = []

# Columns to create edges with ICD9_CODE and DRUG
columns_to_process = ['gender', 'insurance', 'religion', 'marital_status', 'ethnicity', 'age']

# Loop over each column and generate edges
for col in columns_to_process:
    for i in range(len(df)):
        if pd.notnull(df[col].values[i]):  # Ensure the value is not NaN
            # Create edges with ICD9_CODE
            for item in df['icd9_code'].values[i]:
                new_row = {
                    'head': df[col].values[i],
                    'head_type': col,
                    'tail': item,
                    'tail_type': 'disease',
                    'relation': f"{col}_disease"
                }
                rows.append(new_row)  # Store the new row in the list

            # Create edges with DRUG
            if isinstance(df['drug'].values[i], list):
                for drug in df['drug'].values[i]:
                    new_row = {
                        'head': df[col].values[i],
                        'head_type': col,
                        'tail': drug,
                        'tail_type': 'drug',
                        'relation': f"{col}_drug"
                    }
                    rows.append(new_row)  # Store the new row in the list

# Concatenate all rows into mimicKG DataFrame
mimicKG = pd.concat([mimicKG, pd.DataFrame(rows)], ignore_index=True)

# Display the resulting DataFrame
mimicKG


,head,head_type,tail,tail_type,relation
0,m,gender,43889,disease,gender_disease
1,m,gender,78079,disease,gender_disease
2,m,gender,V5843,disease,gender_disease
3,m,gender,25000,disease,gender_disease
4,m,gender,7310,disease,gender_disease
...,...,...,...,...,...
228505,75,age,Acetaminophen,drug,age_drug
228506,75,age,Docusate Sodium,drug,age_drug
228507,75,age,HYDROmorphone-HP,drug,age_drug
228508,75,age,5% Dextrose,drug,age_drug


In [39]:
# create a new column 'weight', if head and tail are the same, add 1 to the weight, remove duplicates
mimicKG['weight'] = 1
mimicKG = mimicKG.groupby(['head', 'head_type', 'tail', 'tail_type', 'relation']).size().reset_index(name='weight')
mimicKG

,head,head_type,tail,tail_type,relation,weight
0,30,age,0388,disease,age_disease,1
1,30,age,1129,disease,age_disease,1
2,30,age,27651,disease,age_disease,1
3,30,age,2859,disease,age_disease,1
4,30,age,30593,disease,age_disease,1
...,...,...,...,...,...,...
38240,widowed,marital_status,sevelamer HYDROCHLORIDE,drug,marital_status_drug,1
38241,widowed,marital_status,traMADOL,drug,marital_status_drug,3
38242,widowed,marital_status,traZODONE,drug,marital_status_drug,18
38243,widowed,marital_status,traZODONE HCl,drug,marital_status_drug,10


In [49]:
# drop rows where weight is less than 2
mimicKG[mimicKG['weight'] >= 3]

,head,head_type,tail,tail_type,relation,weight
647,53,age,3310,disease,age_disease,3
669,53,age,51881,disease,age_disease,3
691,53,age,Acetaminophen,drug,age_drug,3
703,53,age,Bisacodyl,drug,age_drug,3
705,53,age,Calcium Gluconate,drug,age_drug,3
...,...,...,...,...,...,...
38230,widowed,marital_status,Zinc Sulfate,drug,marital_status_drug,5
38231,widowed,marital_status,Zolpidem Tartrate,drug,marital_status_drug,12
38241,widowed,marital_status,traMADOL,drug,marital_status_drug,3
38242,widowed,marital_status,traZODONE,drug,marital_status_drug,18


In [41]:
# get the unique values of relation
mimicKG['relation'].unique()

array(['age_disease', 'age_drug', 'religion_disease', 'religion_drug',
       'ethnicity_disease', 'ethnicity_drug', 'marital_status_drug',
       'marital_status_disease', 'gender_drug', 'gender_disease',
       'insurance_drug', 'insurance_disease'], dtype=object)

In [40]:
# save the mimicKG to a csv
mimicKG.to_csv('mimicKG.csv', index=False)

In [1]:
import pandas as pd

# Load your data
df = pd.read_csv('mid/temp.csv')

# Apply transformations with safeguards for NaN values
df['icd9_code'] = df['icd9_code'].apply(
    lambda x: x[1:-1].split(', ') if isinstance(x, str) else []
)

df['drug'] = df['drug'].apply(
    lambda x: [item.strip() for item in x[1:-1].split(', ')] if isinstance(x, str) else []
)

df['entities'] = df['entities'].apply(
    lambda x: [item.strip() for item in x[1:-1].split(', ')] if isinstance(x, str) else []
)
# strip '
df['icd9_code'] = df['icd9_code'].apply(lambda x: [item.strip().strip("'") for item in x])
df['drug'] = df['drug'].apply(lambda x: [item.strip().strip("'") for item in x])
df['entities'] = df['entities'].apply(lambda x: [item.strip().strip("'") for item in x])

# Display the processed DataFrame
df


,subject_id,gender,insurance,religion,marital_status,ethnicity,icd9_code,drug,age,entities
0,157,m,medicare,catholic,single,white,"[43889, 78079, V5843, 25000, 7310, 2768, 7872,...","[SW, Magnesium Sulfate, Sodium Chloride 0.9% ...",80,"[medical history, tablets, peg tube placement,..."
1,187,f,medicare,protestant quaker,divorced,white,"[0383, 9992, 41071, 496, 2762, 45182, 2768, 33...",[],66,"[primary care physician, blood cultures, drink..."
2,253,f,medicare,catholic,widowed,white,"[4260, 486, 4240, 2449, 3310, 29410, 4168, 272...","[Ipratropium Bromide MDI, Albuterol, Iso-Osmot...",84,"[medical history, discharge, family history, h..."
3,275,m,medicare,unknown,married,white,"[0389, 51881, 78552, 5849, 4538, 2869, 85221, ...","[Cosyntropin, Magnesium Sulfate, Phytonadione,...",82,"[hospital sdh, surgical changes, medical histo..."
4,345,f,medicare,catholic,married,hispanic or latino,"[78039, 42823, 34830, 2762, 2903, 4280, 42731,...","[Metoprolol Tartrate, Furosemide, LeVETiraceta...",89+,"[atrial fibrillation, medical history, family ..."
...,...,...,...,...,...,...,...,...,...,...
681,99417,m,medicare,unknown,married,white,"[0389, 78552, 5570, 51881, 2762, 5609, 99592, ...","[Sodium Chloride 0.9% Flush, 5% Dextrose, Sol...",87,"[medical history, family history, surgical eva..."
682,99439,m,medicare,protestant quaker,married,black/african american,"[34830, 42842, 5849, 4168, 43883, 496, 32723, ...","[Potassium Chloride, 0.9% Sodium Chloride, Iso...",84,"[eye discharge medications, hospice, exuberant..."
683,99454,f,medicare,protestant quaker,married,white,"[41519, 51881, 45342, 2930, 3310, 29410, 2449,...","[Omeprazole, Namenda, Gabapentin, Levothyroxin...",79,"[warfarin 1 mg, department radiology, treat, o..."
684,99864,m,medicare,jewish,single,white,"[5780, 43820, 78791, 7873, V441, 7921, 9916, 2...","[1/2 NS, Insulin, Lisinopril, 0.9% Sodium Chlo...",83,"[primary care physician, calcified granuloma r..."


In [2]:
df1 = df[['icd9_code', 'drug', 'entities']]
df1

,icd9_code,drug,entities
0,"[43889, 78079, V5843, 25000, 7310, 2768, 7872,...","[SW, Magnesium Sulfate, Sodium Chloride 0.9% ...","[medical history, tablets, peg tube placement,..."
1,"[0383, 9992, 41071, 496, 2762, 45182, 2768, 33...",[],"[primary care physician, blood cultures, drink..."
2,"[4260, 486, 4240, 2449, 3310, 29410, 4168, 272...","[Ipratropium Bromide MDI, Albuterol, Iso-Osmot...","[medical history, discharge, family history, h..."
3,"[0389, 51881, 78552, 5849, 4538, 2869, 85221, ...","[Cosyntropin, Magnesium Sulfate, Phytonadione,...","[hospital sdh, surgical changes, medical histo..."
4,"[78039, 42823, 34830, 2762, 2903, 4280, 42731,...","[Metoprolol Tartrate, Furosemide, LeVETiraceta...","[atrial fibrillation, medical history, family ..."
...,...,...,...
681,"[0389, 78552, 5570, 51881, 2762, 5609, 99592, ...","[Sodium Chloride 0.9% Flush, 5% Dextrose, Sol...","[medical history, family history, surgical eva..."
682,"[34830, 42842, 5849, 4168, 43883, 496, 32723, ...","[Potassium Chloride, 0.9% Sodium Chloride, Iso...","[eye discharge medications, hospice, exuberant..."
683,"[41519, 51881, 45342, 2930, 3310, 29410, 2449,...","[Omeprazole, Namenda, Gabapentin, Levothyroxin...","[warfarin 1 mg, department radiology, treat, o..."
684,"[5780, 43820, 78791, 7873, V441, 7921, 9916, 2...","[1/2 NS, Insulin, Lisinopril, 0.9% Sodium Chlo...","[primary care physician, calcified granuloma r..."


In [3]:
# create a list to store all the unique values in the icd9_code, drug, and entities columns
unique_icd9_code = []
unique_drug = []
unique_entities = []

for i in range(len(df1)):
    unique_icd9_code.extend(df1['icd9_code'].values[i])
    unique_drug.extend(df1['drug'].values[i])
    unique_entities.extend(df1['entities'].values[i])

# get the unique values
unique_icd9_code = list(set(unique_icd9_code))
unique_drug = list(set(unique_drug))
unique_entities = list(set(unique_entities))

len(unique_icd9_code), len(unique_drug), len(unique_entities)

(1500, 936, 5249)

In [6]:
# save the list to txt
with open('mapping/mimic_disease.txt', 'w') as f:
    for item in unique_icd9_code:
        f.write("%s\n" % item)

with open('mapping/mimic_drug.txt', 'w') as f:
    for item in unique_drug:
        f.write("%s\n" % item)

In [46]:
import csv
import itertools

new_rows = []
with open('drug_disease_sdoh_combinations.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['drug', 'disease', 'SDoH'])
    for index, row in df1.iterrows():
        drugs = row['drug'] if isinstance(row['drug'], list) else []
        diseases = row['icd9_code'] if isinstance(row['icd9_code'], list) else []
        sdoh_factors = row['entities'] if isinstance(row['entities'], list) else []

        for drug, disease, sdoh in itertools.product(drugs, diseases, sdoh_factors):
            writer.writerow([drug, disease, sdoh])
            new_rows.append([drug, 'associate', disease])
            new_rows.append([disease, 'influence', sdoh])
            #new_rows.append([drug, 'influence', sdoh])

new_df = pd.DataFrame(new_rows, columns=['head', 'relation', 'tail'])
new_df.to_csv('drug_disease_sdoh.csv', index=False)

In [45]:
import csv
import itertools

new_rows = []
with open('drug_disease_sdoh_combinations1.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['drug', 'disease', 'SDoH'])
    for index, row in df1.iterrows():
        drugs = row['drug'] if isinstance(row['drug'], list) else []
        diseases = row['icd9_code'] if isinstance(row['icd9_code'], list) else []
        sdoh_factors = row['entities'] if isinstance(row['entities'], list) else []

        for drug, disease, sdoh in itertools.product(drugs, diseases, sdoh_factors):
            writer.writerow([drug, disease, sdoh])
            #new_rows.append([drug, 'associate', disease])
            #new_rows.append([disease, 'influence', sdoh])
            new_rows.append([drug, 'influence', sdoh])

new_df = pd.DataFrame(new_rows, columns=['head', 'relation', 'tail'])
new_df.to_csv('drug_disease_sdoh1.csv', index=False)

In [46]:
new_df

,head,relation,tail
0,SW,influence,medical history
1,SW,influence,tablets
2,SW,influence,peg tube placement
3,SW,influence,code status
4,SW,influence,ciprofloxacin
...,...,...,...
37538999,Metoprolol Tartrate,influence,follow-up
37539000,Metoprolol Tartrate,influence,iv medications
37539001,Metoprolol Tartrate,influence,medications
37539002,Metoprolol Tartrate,influence,brief hospital


In [47]:
new_df['weight'] = 1
new_df = new_df.groupby(['head', 'relation', 'tail']).size().reset_index(name='weight')
new_df

,head,relation,tail,weight
0,*NF* Bicalutamide,influence,admission,16
1,*NF* Bicalutamide,influence,appointment,16
2,*NF* Bicalutamide,influence,aricept 10 mg,16
3,*NF* Bicalutamide,influence,assisted care facility,16
4,*NF* Bicalutamide,influence,bicalutamide,16
...,...,...,...,...
443177,zinc oxide,influence,treatment,74
443178,zinc oxide,influence,vasopressin,74
443179,zinc oxide,influence,vegetation,74
443180,zinc oxide,influence,warfarin 1 mg,74


In [54]:
# ['head', 'head_type', 'tail', 'tail_type', 'relation', 'weight']
# create new column 'head_type' and 'tail_type'
# if relation is 'associate', head_type = 'drug', tail_type = 'disease'
# if relation is 'influence', head_type = 'disease', tail_type = 'SDoH'
new_df['head_type'] = 'drug'
new_df['tail_type'] = 'disease'
new_df.loc[new_df['relation'] == 'influence', 'head_type'] = 'disease'
new_df.loc[new_df['relation'] == 'influence', 'tail_type'] = 'SDoH'
# change the relation to 'drug_disease' and 'disease_sdoh'
new_df.loc[new_df['relation'] == 'associate', 'relation'] = 'drug_disease'
new_df.loc[new_df['relation'] == 'influence', 'relation'] = 'disease_sdoh'
new_df

,head,relation,tail,weight,head_type,tail_type
0,*NF* Bicalutamide,drug_disease,185,37,drug,disease
1,*NF* Bicalutamide,drug_disease,2749,37,drug,disease
2,*NF* Bicalutamide,drug_disease,2859,37,drug,disease
3,*NF* Bicalutamide,drug_disease,29410,37,drug,disease
4,*NF* Bicalutamide,drug_disease,3310,37,drug,disease
...,...,...,...,...,...,...
492950,zinc oxide,drug_disease,V153,149,drug,disease
492951,zinc oxide,drug_disease,V433,149,drug,disease
492952,zinc oxide,drug_disease,V4986,149,drug,disease
492953,zinc oxide,drug_disease,V5861,149,drug,disease


In [48]:
# ['head', 'head_type', 'tail', 'tail_type', 'relation', 'weight']
# create new column 'head_type' and 'tail_type'
# if relation is 'associate', head_type = 'drug', tail_type = 'disease'
# if relation is 'influence', head_type = 'disease', tail_type = 'SDoH'
new_df['head_type'] = 'drug'
new_df['tail_type'] = 'SDoH'
new_df['relation'] = 'drug_sdoh'
new_df.loc[new_df['relation'] == 'influence', 'relation'] = 'drug_sdoh'
new_df

,head,relation,tail,weight,head_type,tail_type
0,*NF* Bicalutamide,drug_sdoh,admission,16,drug,SDoH
1,*NF* Bicalutamide,drug_sdoh,appointment,16,drug,SDoH
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16,drug,SDoH
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16,drug,SDoH
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16,drug,SDoH
...,...,...,...,...,...,...
443177,zinc oxide,drug_sdoh,treatment,74,drug,SDoH
443178,zinc oxide,drug_sdoh,vasopressin,74,drug,SDoH
443179,zinc oxide,drug_sdoh,vegetation,74,drug,SDoH
443180,zinc oxide,drug_sdoh,warfarin 1 mg,74,drug,SDoH


In [50]:
new_df = new_df[new_df['weight'] >= 10]
new_df

,head,relation,tail,weight,head_type,tail_type
0,*NF* Bicalutamide,drug_sdoh,admission,16,drug,SDoH
1,*NF* Bicalutamide,drug_sdoh,appointment,16,drug,SDoH
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16,drug,SDoH
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16,drug,SDoH
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16,drug,SDoH
...,...,...,...,...,...,...
443177,zinc oxide,drug_sdoh,treatment,74,drug,SDoH
443178,zinc oxide,drug_sdoh,vasopressin,74,drug,SDoH
443179,zinc oxide,drug_sdoh,vegetation,74,drug,SDoH
443180,zinc oxide,drug_sdoh,warfarin 1 mg,74,drug,SDoH


In [51]:
# save the new_df to a csv
#new_df = new_df[new_df['weight'] >= 10]
new_df.to_csv('new_df1.csv', index=False)

In [1]:
import pandas as pd
import numpy as np

new_df = pd.read_csv('new_df.csv')
mimicKG = pd.read_csv('mimicKG.csv')

In [52]:
mimicKG = pd.read_csv('mimicKG.csv')
new_df1 = pd.read_csv('new_df1.csv')

In [53]:
# merge the two dataframes
final_df = pd.concat([new_df, mimicKG], ignore_index=True)
final_df

,head,relation,tail,weight,head_type,tail_type
0,*NF* Bicalutamide,drug_sdoh,admission,16,drug,SDoH
1,*NF* Bicalutamide,drug_sdoh,appointment,16,drug,SDoH
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16,drug,SDoH
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16,drug,SDoH
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16,drug,SDoH
...,...,...,...,...,...,...
923384,widowed,marital_status_drug,Vitamin D,60,marital_status,drug
923385,widowed,marital_status_drug,Warfarin,41,marital_status,drug
923386,widowed,marital_status_drug,Zolpidem Tartrate,12,marital_status,drug
923387,widowed,marital_status_drug,traZODONE,18,marital_status,drug


In [54]:
# get the unique values of relation
final_df['relation'].unique()

array(['drug_sdoh', 'drug_disease', 'disease_sdoh', 'age_disease',
       'age_drug', 'ethnicity_drug', 'ethnicity_disease', 'religion_drug',
       'religion_disease', 'marital_status_disease',
       'marital_status_drug', 'gender_drug', 'gender_disease',
       'insurance_drug', 'insurance_disease'], dtype=object)

In [55]:
# save the final_df to a csv
final_df.to_csv('final_df.csv', index=False)

In [56]:
df = final_df[final_df['weight'] >= 10]
# save the final_df to a csv
df.to_csv('mimicKG.csv', index=False)